In [2]:
#Import the libraries
from textblob import TextBlob
import nltk
import pandas as pd
from textblob import TextBlob
import pandas as pd
import string
import re
from string import digits
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
import gensim
import matplotlib.pyplot as plt
from stemming.porter2 import stem

In [2]:
brexit = pd.read_csv( 'brexit.csv')
brexit = brexit.dropna()

In [3]:
brexit = brexit[brexit['comments'] != '[deleted]']

In [4]:
def preprocessing(x):
    x = x.lower()
#     x = x.replace('—', ' ')
    x = x.replace('-', ' ')
    x = re.sub('\w+@\w+\.[a-z]{3}', ' ', x)
    x = x.translate(str.maketrans(" ", " ", string.punctuation))
    x = x.translate(str.maketrans('', '', digits))
    x = x.replace('“', '')
    x = x.replace('”', '')
    x = x.replace('…', '')
    x = x.replace('’', '')
    x = x.replace('‘', '')  
    x = ' '.join(x.split())
    x = x.split('—')
#     print(x)
    if(x[0].isupper()):
        x = x[1:]
    x = ' '.join(x)
    x = x.strip()
    x = x.replace('  ', '')
    return x
brexit['comments'].apply(preprocessing)

0         cheers ya colossal floppy haired bucket of bad...
1         love the fact both deals are negative seriousl...
2         he doesnt give a shit about the british people...
3         dont worry the dup is already coming out in op...
4         name one deal where the british people dont en...
5         but not him and his hedge fund buddies so who ...
6         as a uk citizen we were made aware that leavin...
7         thats by design the rich need to keep their sl...
8          brexiteers cheer at the thought of economic ruin
9         dont worry itll still make a specific subset o...
10        glad to see the usa isnt the only country who ...
11        yeah but its only for the next years nothing t...
12                          they should just revoke article
13        remainer here with a quiet reminder that aroun...
14        does anybody still doubt that brexit is an eco...
15        it will make some british even more wealthy wh...
16        to be honest any deal will mak

In [6]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

brexit['comments'].apply(get_stemmed_text)
brexit.head()

,comments
0,"Cheers ya colossal, floppy haired bucket of ba..."
1,"Love the fact both deals are negative, serious..."
2,He doesn’t give a shit about the British peopl...
3,"Don't worry, the DUP is already coming out in ..."
4,Name one deal where the British people don't e...


In [7]:
comments = brexit['comments']
counter = 0
positive_sentiment_array = []
negative_sentiment_array = []
sentiment_score_array = []
label_array = []
topic_array = []
for comment in comments:
    counter += 1
    obj = TextBlob( comment )
    sentiment = obj.sentiment.polarity
    if sentiment > 0.0:
            positive_sentiment_array.append(obj)
            label_array.append("positive")
    if sentiment <0.0:
            negative_sentiment_array.append(obj)
            label_array.append("negative")
    if sentiment == 0.0:
            label_array.append('neutral')
    sentiment_score_array.append(sentiment)
    topic_array.append("Brexit")
    
#returns the sentiment of text
#by returning a value between -1.0 and 1.0

print(len(positive_sentiment_array))
print(len(negative_sentiment_array))
print(counter)

93354
53002
203125


In [9]:
brexit.insert(1, "Score", sentiment_score_array , True) 
brexit.insert(2, "Label", label_array , True) 
brexit.insert(3, "Topic", topic_array , True)    
brexit

,comments,Score,Label,Topic
0,"Cheers ya colossal, floppy haired bucket of ba...",0.550000,positive,Brexit
1,"Love the fact both deals are negative, serious...",-0.044444,negative,Brexit
2,He doesn’t give a shit about the British peopl...,-0.074219,negative,Brexit
3,"Don't worry, the DUP is already coming out in ...",-0.166667,negative,Brexit
4,Name one deal where the British people don't e...,-0.050000,negative,Brexit
5,But not him and his hedge fund buddies so who ...,-0.400000,negative,Brexit
6,As a Uk citizen we were made aware that leavin...,-0.272857,negative,Brexit
7,That's by design. The rich need to keep their ...,0.375000,positive,Brexit
8,Brexiteers cheer at the thought of economic ruin,0.200000,positive,Brexit
9,Don't worry. It'll still make a specific subse...,0.470000,positive,Brexit


In [12]:
brexit_sentiment.to_csv('Brexit_SentimentAnalysis.csv',encoding='utf-8-sig')

In [13]:
brexit_sentiment = pd.read_csv('../Individual_SentimentAnalysis_Files/Brexit_SentimentAnalysis.csv')

In [20]:
brexit_sentiment

,Unnamed: 0,comments,Score,Label,Topic
0,0,"Cheers ya colossal, floppy haired bucket of ba...",0.55,positive,Brexit
1,1,"Love the fact both deals are negative, serious...",-0.0444444,negative,Brexit
2,2,He doesn’t give a shit about the British peopl...,-0.0742188,negative,Brexit
3,3,"Don't worry, the DUP is already coming out in ...",-0.166667,negative,Brexit
4,4,Name one deal where the British people don't e...,-0.05,negative,Brexit
5,5,But not him and his hedge fund buddies so who ...,-0.4,negative,Brexit
6,6,As a Uk citizen we were made aware that leavin...,-0.272857,negative,Brexit
7,7,That's by design. The rich need to keep their ...,0.375,positive,Brexit
8,8,Brexiteers cheer at the thought of economic ruin,0.2,positive,Brexit
9,9,Don't worry. It'll still make a specific subse...,0.47,positive,Brexit


In [21]:
brexit_sentiment['Score'] = brexit_sentiment.Score.convert_objects(convert_numeric=True)
max_score = brexit_sentiment['Score'].max()
min_score = brexit_sentiment['Score'].min()

/Users/yuankanglee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [22]:
brexit_sentiment['normalized_score'] = brexit_sentiment.apply(
lambda x: (x['Score'] - min_score)/(max_score-min_score), axis=1)

In [23]:
brexit_sentiment['normalized_score']

0         0.775000
1         0.477778
2         0.462891
3         0.416667
4         0.475000
5         0.300000
6         0.363571
7         0.687500
8         0.600000
9         0.735000
10        0.642857
11        0.500000
12        0.500000
13        0.459896
14        0.609524
15        0.575000
16        0.608333
17        0.642857
18        0.612500
19        0.596875
20        0.750000
21        0.562500
22        0.687500
23        0.500000
24        0.687500
25        0.500000
26        0.500000
27        0.500000
28        0.500000
29        1.000000
            ...   
349565    0.595833
349566    0.489583
349567    0.500000
349568    0.354167
349569    0.687500
349570    0.800000
349571    0.250000
349572    0.100000
349573    0.516250
349574    0.478333
349575    0.508750
349576    0.567118
349577    0.379167
349578    0.675000
349579    0.800000
349580    0.500000
349581    0.500000
349582    0.712500
349583    0.622222
349584    0.675000
349585    0.500000
349586    0.

In [25]:
def positive_or_negative(x):
    if x > 0.5:
        return 'postive'
    elif x < 0.5:
        return 'negative'
    elif x == 0.5:
        return 'neutral'

brexit_sentiment['normalized_pos_neg'] = brexit_sentiment.apply(
lambda x: positive_or_negative(x['normalized_score']) , axis=1)

In [26]:
brexit_sentiment

,Unnamed: 0,comments,Score,Label,Topic,normalized_score,normalized_pos_neg
0,0,"Cheers ya colossal, floppy haired bucket of ba...",0.550000,positive,Brexit,0.775000,postive
1,1,"Love the fact both deals are negative, serious...",-0.044444,negative,Brexit,0.477778,negative
2,2,He doesn’t give a shit about the British peopl...,-0.074219,negative,Brexit,0.462891,negative
3,3,"Don't worry, the DUP is already coming out in ...",-0.166667,negative,Brexit,0.416667,negative
4,4,Name one deal where the British people don't e...,-0.050000,negative,Brexit,0.475000,negative
5,5,But not him and his hedge fund buddies so who ...,-0.400000,negative,Brexit,0.300000,negative
6,6,As a Uk citizen we were made aware that leavin...,-0.272857,negative,Brexit,0.363571,negative
7,7,That's by design. The rich need to keep their ...,0.375000,positive,Brexit,0.687500,postive
8,8,Brexiteers cheer at the thought of economic ruin,0.200000,positive,Brexit,0.600000,postive
9,9,Don't worry. It'll still make a specific subse...,0.470000,positive,Brexit,0.735000,postive


In [27]:
brexit_sentiment['normalized_score'].mean()

0.5255899790392367